In [5]:
# As usual, a bit of setup
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

1 + 1

2

In [16]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 500, 500, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def style_model(img, y):
    """Generate class scores for img
    
    Inputs:
    - img: Tensor of image [batch_size, 500, 500, 3]
    - y: class [batch_size, 20]
    
    Returns:
    Class scores [batch_size, 20].
    """
    
    dims = tf.shape(img)
    batch_size = dims[0]
    
    conv1 = tf.layers.conv2d(
        inputs=img,
        filters=32,
        kernel_size=[5, 5],
        padding="valid",
        activation=tf.nn.relu)

    print(conv1)
    
    flat = tf.reshape(conv1, [batch_size, 7872512]) #496 * 496 * 32
    
    print(flat)
    
    dense = tf.layers.dense(inputs=flat, units=20, activation=tf.nn.relu)

    return dense

y_out = style_model(X, y)

# define our loss
total_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)


Tensor("conv2d/Relu:0", shape=(?, 496, 496, 32), dtype=float32)
Tensor("Reshape:0", shape=(?, 7872512), dtype=float32)
